In [24]:
# import libraries
import subprocess
import json
import os

from constants import *
from dotenv import load_dotenv
from pathlib import Path
from pprint import pprint
from web3 import Web3
from eth_account import Account
from bit import PrivateKeyTestnet
from bit.network import NetworkAPI
from web3.middleware import geth_poa_middleware
from web3.gas_strategies.time_based import medium_gas_price_strategy

load_dotenv()

True

In [25]:
# import mnemonic from env
filepath=".env"
p = subprocess.Popen(filepath, shell=True, stdout = subprocess.PIPE)

#get mnemonic
mnemonic = os.getenv('MNEMONIC')
print(mnemonic)

stumble sing material day acoustic try page jungle mosquito service grocery century


In [26]:
# connect to local ETH/geth
w3 = Web3(Web3.HTTPProvider('http://localhost:8545'))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)
w3.eth.setGasPriceStrategy(medium_gas_price_strategy)

In [27]:
# Define function to derive wallet
def derive_wallets(mnemonic, coin, numderive):
    """Use the subprocess library to call the php file script from Python"""
    command = f'php ./hd-wallet-derive/hd-wallet-derive.php -g --mnemonic="{mnemonic}" --numderive="{numderive}" --coin="{coin}" --cols=path,address,privkey,pubkey --format=json' 
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
    p_status = p.wait()
    keys = json.loads(output)
    return  keys

In [28]:
# Created a dictionary object called coins to store the output from `derive_wallets`.
coins = {"eth", "btc", "btc-test"}
print(coins)
numderive = 3

{'eth', 'btc', 'btc-test'}


In [29]:
key = {}
for coin in coins:
    key[coin]= derive_wallets(os.getenv('mnemonic'), coin, numderive=3)

In [30]:

eth_PrivateKey = key["eth"][0]['privkey']
btc_PrivateKey = key["btc-test"][0]['privkey']

print(json.dumps(eth_PrivateKey, indent=4, sort_keys=True))
print(json.dumps(btc_PrivateKey, indent=4, sort_keys=True))

"0x883a22ca435f5ca2a37a4f587983f455d25f6cf2bd92529278485a25a7113f20"
"cR38nnMUzPKg7TQSeQ7sueEJBbQiMUViyj8UAamVJ2JRexMqHfgL"


In [31]:
print(json.dumps(key, indent=4, sort_keys=True))

{
    "btc": [
        {
            "address": "12jYeEnU3wdwEMogfG6FofK6pe9PysiZAc",
            "path": "m/44'/0'/0'/0/0",
            "privkey": "KynzJmCL5twkxYprUuhAtSiRtWHUzT9soBHrvpMSkqfMqnBcBJ8p",
            "pubkey": "037d1c6bce0cffdbdd576f71a980b6928f119287077854a9d2516e43d385bedff5"
        },
        {
            "address": "1Gs5KGy6wnjxXmS9CdxagNXnftUM6bKWsp",
            "path": "m/44'/0'/0'/0/1",
            "privkey": "KxQtFqLLVeXNSA6XAQgi6gX7jLJHuwguAZW5tJszPzNXrc335a4K",
            "pubkey": "02fce3efd13d07c23d17ac45f080e72b68c21371019b2a3cec5e29e77b77baab13"
        },
        {
            "address": "1DQRvfasJzoirRcyS9KnPsGn7b6S8LNmno",
            "path": "m/44'/0'/0'/0/2",
            "privkey": "L3tx53tWgHZuHHfzFu3jfNt5vjTM7MngB61DryNf8zdFiK7dNLos",
            "pubkey": "03e8dc22cf608c2c79d9fc4ecf5689e556b1a51b2dee5c90105c587a215122139d"
        }
    ],
    "btc-test": [
        {
            "address": "mzKe2XK1Vb4Cxnxi2h6N3g9pxGUppTUCnm",
            "path

In [32]:
# Created a function called `priv_key_to_account` that converts privkey strings to account objects.

def priv_key_to_account(coin,priv_key):
    print(coin)
    print(priv_key)
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    elif coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)

In [33]:
# Created a function called `create_tx` that creates an unsigned transaction appropriate metadata.
def create_tx(coin,account, recipient, amount):
    if coin == ETH: 
        gasEstimate = w3.eth.estimateGas(
            {"from":eth_acc.address, "to":recipient, "value": amount}
        )
        return { 
            "from": eth_acc.address,
            "to": recipient,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(eth_acc.address)
        }
    
    elif coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])

In [34]:
# Created a function called `send_tx` that calls `create_tx`, signs and sends the transaction.
def send_txn(coin,account,recipient, amount):
    txn = create_tx(coin, account, recipient, amount)
    if coin == ETH:
        signed_txn = eth_acc.sign_transaction(txn)
        result = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
        print(result.hex())
        return result.hex()
    elif coin == BTCTEST:
        tx_btctest = create_tx(coin, account, recipient, amount)
        signed_txn = account.sign_transaction(txn)
        print(signed_txn)
        return NetworkAPI.broadcast_tx_testnet(signed_txn)

In [35]:
# BTC-TEST 
btc_acc = priv_key_to_account(BTCTEST,btc_PrivateKey)

btc-test
cR38nnMUzPKg7TQSeQ7sueEJBbQiMUViyj8UAamVJ2JRexMqHfgL


In [44]:
create_tx(BTCTEST,btc_acc,"mzKe2XK1Vb4Cxnxi2h6N3g9pxGUppTUCnm", 0.0001)

'{"unspents":[{"amount":1386102,"confirmations":1,"script":"76a914ce4690ad6e9cb9dd3cc0c9ffe9ff987ff7bc8a1888ac","txid":"ba6318d779ef2c197e097c3b7ef369a1fbf2f5d9b561db73ac553efaf1508281","txindex":0,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295},{"amount":10000,"confirmations":5,"script":"76a914ce4690ad6e9cb9dd3cc0c9ffe9ff987ff7bc8a1888ac","txid":"c1bd16b5eb55c619229e8711146014213c01429d15280bd54400f44489c7f9e4","txindex":0,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295},{"amount":10000,"confirmations":2,"script":"76a914ce4690ad6e9cb9dd3cc0c9ffe9ff987ff7bc8a1888ac","txid":"53e75338693e49d9e948b8ffcff2726e29c8a71b22737ce131b10eea43a2f43e","txindex":1,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295}],"outputs":[["mzKe2XK1Vb4Cxnxi2h6N3g9pxGUppTUCnm",10000],["mzKe2XK1Vb4Cxnxi2h6N3g9pxGUppTUCnm",1342858]]}'

In [42]:
# Send BTC Transaction
send_txn(BTCTEST,btc_acc,"mzKe2XK1Vb4Cxnxi2h6N3g9pxGUppTUCnm", 0.00001)

0100000003e4f9c78944f40044d50b28159d42013c2114601411879e2219c655ebb516bdc1000000006a47304402207764b5085f753a6a37376c8a2172bb36a18ee83324ffd81de7f92559e47bd06302206e0d0bf20ff5a526117eb11b37bd493b0ae1124c89d2369cb928507130c10650012103a17f7f6f8d50f320920496320dca80e977ec317e35917f971587579b31131574ffffffff3ef4a243ea0eb131e17c73221ba7c8296e72f2cfffb848e9d9493e693853e753010000006a473044022018418346a3033d5a312824a87bd930ec5c415fa9f848d5a8a1ee58b680bf2de8022079557e8c6719be652efe0956d273d28e25a95159298496327d7671874df5799e012103a17f7f6f8d50f320920496320dca80e977ec317e35917f971587579b31131574ffffffff818250f1fa3e55ac73db61b5d9f5f2fba169f37e3b7c097e192cef79d71863ba000000006b483045022100dca505bf2e3af32b10a0d8469fd33b5d1105c51737dff4bc7a018d61761971c6022035891da73321219682c6f8df4165411a7370839521a9c341133cfbc3698c5033012103a17f7f6f8d50f320920496320dca80e977ec317e35917f971587579b31131574ffffffff02e8030000000000001976a914ce4690ad6e9cb9dd3cc0c9ffe9ff987ff7bc8a1888acb2a01400000000001976a914ce4690ad6e9c

In [37]:
# Connecting to HTTP with address
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))

In [38]:

# Connection Test
w3.isConnected()

True

In [39]:
def create_raw_tx(account, recipient, amount):
    gasEstimate = w3.eth.estimateGas(
        {"from": account.address, "to": recipient, "value": amount}
    )
    return {
        "from": account.address,
        "to": recipient,
        "value": amount,
        "gasPrice": w3.eth.gasPrice,
        "gas": gasEstimate,
        "nonce": w3.eth.getTransactionCount(account.address),
    }


In [40]:

def send_tx(account, recipient, amount):
    tx = create_raw_tx(account, recipient, amount)
    signed_tx = account.sign_transaction(tx)
    result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
    print(result.hex())
    return result.hex()

In [41]:
# Confirmation of Transactional

w3.eth.getBalance("0x806647dAa1C656B8aEC45408E7B850BA88b14C3F")

#0xbEC0d63779f5d59346a135cCf681bFa5B13d61C8
#0x8aD5b4E4165647E4666587D8f4b66f268F46F772


31999580000000000000